# 1. Connect to IB 

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:

from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

# 2. Market Segment Analysis

In [3]:
import market_analyzer as ma

# Initialize analyzer
analyzer = ma.IBMarketAnalyzer(ib)

# IMPORTANT: Set up the sectors first
analyzer.set_sectors()  # This was missing in the previous code

# Now get the full analysis
market_df, sector_df = analyzer.create_analysis_report()



Running market and sector analysis first...


In [4]:


# Then filter the sectors in various ways:

custom_filter_df = analyzer.filter_sectors( 
    sector_df,
    as_list_of_sectors=False,
    min_momentum=0,
    sort_by='Overall Score',
    min_score=5
)

custom_filter_list = analyzer.filter_sectors( 
    sector_df,
    as_list_of_sectors=True,
    min_momentum=0,
    sort_by='Overall Score',
    min_score=5
)

# ib.disconnect()

print("=== Market Analysis ===")
display(market_df)

print("\n=== Sector Analysis ===")
display(sector_df)


try:


    print("\n=== Custom Filter DF ===")
    display(custom_filter_df)

    print("\n=== Custom Filter Sector List ===")
    display(custom_filter_list)

except Exception as e:
    print(f"An error occurred: {str(e)}")

status_dict = {
    "sector_analysis": len(sector_df),
    "stocks_scanned": 0,
    "stocks_validated": 0,
    "stocks_scored": 0
}

# ------------------- EMAIL -------------------
from emails import email_summery, email_client

# Create the email generator and generate email
email_generator = email_summery.StockSummaryEmail()
email_html = email_generator.generate_email(
    status_dict,
    scan_settings= None,
    activity_df = None,
    sector_etf_df = sector_df.round(2),
    fundamentals_df = None,
    ta_df = None,
    daily_ta_df = None
)

email_client.send_outlook_email(
    subject="STOCKBOT: Daily Stock Summery",
    body=email_html,
    image_paths=[],
    recipients=['pary888@gmail.com'],
    is_html=True)

=== Market Analysis ===


,Metric,Description,Value,Interpretation
0,SPY Current Price,Current market price of SPY ETF,606.79,Current market price point
1,SPY 20-Day MA,20-day simple moving average - short-term trend indicator,600.88,Price above 20MA is bullish short-term
2,SPY 50-Day MA,50-day simple moving average - medium-term trend indicator,589.87,Price above 50MA is bullish medium-term
3,SPY 200-Day MA,200-day simple moving average - long-term trend indicator,549.67,Price above 200MA is bullish long-term
4,SPY VWAP,Volume Weighted Average Price - shows average price based on volume,526.1,Price above VWAP indicates buying pressure
5,Price vs 20MA (%),Percentage difference between current price and 20-day MA,0.98,Positive % indicates price strength vs 20MA
6,Price vs 50MA (%),Percentage difference between current price and 50-day MA,2.87,Positive % indicates price strength vs 50MA
7,Price vs 200MA (%),Percentage difference between current price and 200-day MA,10.39,Positive % indicates price strength vs 200MA
8,Price vs VWAP (%),Percentage difference between current price and VWAP,15.34,Positive % indicates current buying pressure
9,SPY RSI,Relative Strength Index (14-day) - momentum indicator (0-100),64.74,"RSI > 70 overbought, < 30 oversold"



=== Sector Analysis ===


,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
10,Energy,XLE,-9.09,-12.26,-7.68,Increasing,No,Negative,11,11.0,11.0,-1.000000
9,Materials,XLB,-3.86,-7.03,-6.63,Increasing,No,Negative,10,10.0,10.0,0.000000
8,Utilities,XLU,-3.65,-6.82,-5.09,Decreasing,No,Negative,9,9.0,8.0,1.333333
7,Healthcare,XLV,-1.85,-5.02,-5.33,Increasing,No,Negative,8,8.0,9.0,1.666667
6,Real_Estate,XLRE,-1.73,-4.90,-3.86,Decreasing,No,Negative,7,7.0,7.0,3.000000
5,Financials,XLF,-1.14,-4.31,-2.71,Decreasing,No,Negative,6,6.0,5.0,4.333333
4,Industrials,XLI,-0.99,-4.16,-3.76,Increasing,No,Negative,5,5.0,6.0,4.666667
3,Consumer_Staples,XLP,1.35,-1.82,-1.82,Decreasing,No,Negative,4,4.0,4.0,6.000000
2,Technology,XLK,5.23,2.06,2.33,Increasing,Yes,Positive,3,3.0,2.0,7.333333
1,Communication Services,XLC,5.45,2.28,2.14,Increasing,Yes,Positive,2,2.0,3.0,7.666667



=== Custom Filter DF ===


,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
0,Consumer_Discretionary,XLY,10.80,7.63,6.44,Increasing,Yes,Positive,1,1.0,1.0,9.000000
1,Communication Services,XLC,5.45,2.28,2.14,Increasing,Yes,Positive,2,2.0,3.0,7.666667
2,Technology,XLK,5.23,2.06,2.33,Increasing,Yes,Positive,3,3.0,2.0,7.333333



=== Custom Filter Sector List ===


(['Consumer_Discretionary', 'Communication Services', 'Technology'],
 ['XLY', 'XLC', 'XLK'])

True

# 3. Scanner

In [5]:
import scanner
import schedule
from datetime import datetime

market = schedule.MarketSchedule(scan_time='09:00')
sbscan = scanner.StockbotScanner(ib)

# if market.wait_for_scan_time():
#     sbscan.multiscan(scan_code='TOP_PERC_GAIN', price=(1, 100), volume=100_000, change_perc=4, market_cap=100)


# Scan settings so it can be logged and sent via email.
scan_settings = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "scan_code": "TOP_PERC_GAIN",
    "price_min": 1,
    "price_max": 100,
    "volume": 100_000,  # Will be formatted as "100K"
    "market_cap": 100_000_000,  # Will be formatted as "100M"
    "change_percent": 4,
    "total_scanned": 0
}

if market.wait_for_scan_time():
    sbscan.multiscan(
        scan_code=scan_settings['scan_code'], 
        price=(scan_settings['price_min'], scan_settings['price_max']),
        volume=scan_settings['volume'], 
        change_perc=4, 
        market_cap=scan_settings['market_cap']/1_000_000
    )

sbscan.scan_results_df

scan_settings['total_scanned'] = len(sbscan.scan_results_df)
scan_settings

# ------------------- EMAIL -------------------
from emails import email_summery, email_client

status_dict['stocks_scanned'] = scan_settings['total_scanned']

# Create the email generator and generate email
email_generator = email_summery.StockSummaryEmail()
email_html = email_generator.generate_email(
    status_dict,
    scan_settings= scan_settings,
    activity_df = None,
    sector_etf_df = sector_df.round(2),
    fundamentals_df = None,
    ta_df = None,
    daily_ta_df = None
)

email_client.send_outlook_email(
    subject="STOCKBOT: Daily Stock Summery",
    body=email_html,
    image_paths=[],
    recipients=['pary888@gmail.com'],
    is_html=True)

Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (100.0, 1000.0)
STK.US.MAJOR .. len(data): 1
Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (1000.0, 10000.0)
STK.US.MAJOR .. len(data): 0
Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (10000.0, 100000.0)
STK.US.MAJOR .. len(data): 0


Error 162, reqId 105: Historical Market Data Service error message:API scanner subscription cancelled: 105
Error 162, reqId 106: Historical Market Data Service error message:API scanner subscription cancelled: 106
Error 162, reqId 107: Historical Market Data Service error message:API scanner subscription cancelled: 107


,rank,market_cap_range,symbol,exchange,currency,primaryExchange
0,0,"(100.0, 1000.0)",LAES,SMART,USD,


True

# 4. Daily Stock Validation (Fundamentals & TA)

In [3]:
import stock
from strategies import ta
import pandas as pd
import scanner
import schedule


sbscan = scanner.StockbotScanner(ib)

# getting the stock data
# allowed_etfs = ['XLK', 'XLY', 'XLI']

# scan_results_df = pd.DataFrame({
#     'rank': [1, 2, 3],
#     'symbol': ['TSLA', 'AAPL', 'MSFT'],
# })



# permisaable ETFs
allowed_etfs = custom_filter_list[1]
print(f"Allowed ETFs: {allowed_etfs}")



# see how the scans are validatd here: C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\stock.py --> get_funadmentals_validation_results(), get_TA_validation_results()
updated_scan_results_df = sbscan.update_scan_results(sbscan.scan_results_df, ['XLK', 'XLY', 'XLI'], ib, send_email_if_any=False)
updated_scan_results_df

status_dict['stocks_validated'] = len(sbscan.fund_results_df['Fundamentals Passed']) + len(sbscan.ta_results_df['TA Passed'])

# ------------------- EMAIL -------------------
from emails import email_summery, email_client

# Create the email generator and generate email
email_generator = email_summery.StockSummaryEmail()
email_html = email_generator.generate_email(
    status_dict,
    scan_settings   = scan_settings,
    sector_etf_df   = sector_df.round(2),
    fundamentals_df = sbscan.fund_results_df,
    ta_df           = sbscan.ta_results_df,
    daily_ta_df = None,
    activity_df = None,
)

email_client.send_outlook_email(
    subject="STOCKBOT: Daily Stock Summery",
    body=email_html,
    image_paths=[],
    recipients=['pary888@gmail.com'],
    is_html=True)

Using cached data for TSLA (age: 2024-12-15 11:54:51)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TSLA_1_day.csv
Chart saved as C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\fundamental_data_store\TSLA.png
Zoomed chart saved as C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\fundamental_data_store\TSLA_zoomed.png
Email sent to for TSLA
Using cached data for AAPL (age: 2024-12-14 09:55:07)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\AAPL_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\AAPL_1_day.csv
Chart saved as C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\fundamental_data_store\AAPL.png
Zoomed chart saved as C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\fundamental_data_store\AAPL_zoomed.png
Email sent to for AAPL
Using cached data 

,rank,symbol,Sector1 Valid,Sector2 Valid,Market Cap > 300M,Vol 10DayMA > 300K,Fundamentals Passed,Close > $1,Above 200MA,Above 150MA,Breaks Above 50MA,50MA Slope > 0,Gap Up > 4%,Volume > 50K,Volume Above 10MA,Volume Dev > 80%,TA Passed
0,1,TSLA,True,True,True,True,True,True,True,True,False,True,True,True,False,False,False
1,2,AAPL,True,True,True,True,True,True,True,True,False,True,False,True,False,False,False
2,3,MSFT,True,False,True,True,False,True,True,True,False,True,False,True,False,False,False


In [6]:
sbscan.daily_stockx[0].image_path

{'valid_fundamentals_df': 'C:\\Users\\sidsu\\anaconda3\\envs\\SB4\\stockbot4\\data\\fundamental_data_store\\TSLA_fundamentals.png',
 'valid_ta_df': 'C:\\Users\\sidsu\\anaconda3\\envs\\SB4\\stockbot4\\data\\fundamental_data_store\\TSLA_ta.png',
 'chart': 'C:\\Users\\sidsu\\anaconda3\\envs\\SB4\\stockbot4\\data\\fundamental_data_store\\TSLA.png',
 'zoomed_chart': 'C:\\Users\\sidsu\\anaconda3\\envs\\SB4\\stockbot4\\data\\fundamental_data_store\\TSLA_zoomed.png'}

# 5. Daily Stock Scores (TA)

In [1]:
import os

# Go up one level from scanner folder, then into data folder
current_dir = os.getcwd()  # Gets current directory (scanner)
parent_dir = os.path.dirname(current_dir)  # Goes up one level to stockbot4
data_folder = os.path.join(parent_dir, 'data')  # Goes into data folder
os.chdir(data_folder)

import historical_data as hd

symbol = 'ROKU'
interval = '1 day'
# data = hd.get_hist_data(symbol, '52 weeksAgo', 'now', interval)
data = hd.get_hist_data(symbol, '2022-12-30', '2024-11-14', interval)
display(data)

Loading data from historical_data/ROKU_1_day.csv
Stored data: 478 rows of data
Loading data from historical_data/ROKU_1_day.csv


,open,high,low,close,volume
date,,,,,
2022-12-30,40.30,40.97,39.05,40.75,42515
2023-01-03,41.45,42.37,39.00,40.51,51584
2023-01-04,40.93,43.28,39.84,42.47,68893
2023-01-05,42.59,44.25,40.93,42.81,78793
2023-01-06,42.75,44.46,41.30,43.60,56965
...,...,...,...,...,...
2024-11-08,71.06,74.14,70.76,73.98,30291
2024-11-11,74.61,77.21,73.85,77.00,25805
2024-11-12,77.12,77.74,75.55,76.30,20699


# 6. Daily TA

# 7. Filter TA

In [7]:
# BreaksMA = 200 // yesterday below,  today above
# BreaksMA = 50 // yesterday below,  today above
# > MA200
# BreaksVolumeMA = 10 // breaks 10 day vol ma
# MA50isRisingNthDays = (3, 10) // viable range midpoint is best score
# With Segment trend
# With Mkt trend 


ta_1 = ta.Breaks('close', 'MA_cl_200', True) # BreaksMA = 200
ta_2 = ta.Breaks('close', 'MA_cl_50', True) # BreaksMA = 50
ta_3 = ta.AboveBelow('close', 'MA_cl_50', True) # > MA50
ta_4 = ta.AboveBelow('close', 'MA_cl_200', True) # > MA200
ta_5 = ta.AboveBelow(3, 'TDUR_MA_cl_50', True) # MA50isRisingNthDays
ta_6 = ta.AboveBelow(80, 'VDEV_10', True) # > 80% above MA10

f.add_ta(ta_1)
f.add_ta(ta_2)
f.add_ta(ta_3)
f.add_ta(ta_4)
f.add_ta(ta_5)
f.add_ta(ta_6)

ta_filter_names = [ta_1.name, ta_2.name, ta_3.name, ta_4.name, ta_5.name, ta_6.name]

df = f.data.copy()

# add column to sum up the scores
df['filter_score'] = df[ta_filter_names].sum(axis=1)
df

# filter out the rows with score less than 3
filtered_df = df[df['filter_score'] >= 5]
filtered_df

f.load_ohlcv(data)
f.update_ta_data()
f.data

             open   high    low  close  volume
date                                          
2022-12-30  40.30  40.97  39.05  40.75   42515
2023-01-03  41.45  42.37  39.00  40.51   51584
2023-01-04  40.93  43.28  39.84  42.47   68893
2023-01-05  42.59  44.25  40.93  42.81   78793
2023-01-06  42.75  44.46  41.30  43.60   56965
...           ...    ...    ...    ...     ...
2024-11-08  71.06  74.14  70.76  73.98   30291
2024-11-11  74.61  77.21  73.85  77.00   25805
2024-11-12  77.12  77.74  75.55  76.30   20699
2024-11-13  76.25  78.57  75.10  75.31   20926
2024-11-14  75.30  76.35  74.20  74.26   17070

[472 rows x 5 columns]


,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,,,,,,,,,,,
2022-12-30,40.30,40.97,39.05,40.75,42515,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-03,41.45,42.37,39.00,40.51,51584,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-04,40.93,43.28,39.84,42.47,68893,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-05,42.59,44.25,40.93,42.81,78793,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
2023-01-06,42.75,44.46,41.30,43.60,56965,NaN,NaN,NaN,NaN,NaN,1,False,False,False,True,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-08,71.06,74.14,70.76,73.98,30291,65.92035,73.2768,48936.0,3.54,-38.100785,4,True,True,True,False,True,NaN,NaN
2024-11-11,74.61,77.21,73.85,77.00,25805,65.86085,73.4626,50131.6,22.44,-48.525481,5,False,True,True,False,True,NaN,NaN
2024-11-12,77.12,77.74,75.55,76.30,20699,65.77455,73.6870,50755.3,36.49,-59.218052,6,False,True,True,False,True,NaN,NaN


# 8. Daily Scores

In [8]:
df = f.data.copy()

# add column to sum up the scores
df['filter_score'] = df[ta_filter_names].sum(axis=1)
df

# filter out the rows with score less than 3
filtered_df = df[df['filter_score'] >= 5]
filtered_df

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,,,,,,,,,,,
2023-06-01,58.13,59.27,56.48,58.88,30579,NaN,58.8208,42705.7,7.60,-28.395975,-25,True,True,False,True,True,5,NaN
2023-11-02,70.27,79.12,69.00,77.57,332103,66.79200,71.4652,77477.6,24.07,328.643892,-21,True,True,True,True,False,5,NaN
2024-01-08,88.90,94.90,88.85,93.56,55274,74.14190,90.6266,42749.5,-18.88,29.297419,39,True,True,True,False,True,5,NaN
2024-02-07,92.89,96.06,91.87,94.75,39558,76.91170,94.0274,34138.4,7.47,15.875378,-7,True,True,True,True,True,6,NaN
2024-05-17,61.50,61.86,60.60,61.45,14528,76.73030,61.3974,27163.7,9.47,-46.516859,-77,True,True,False,True,True,5,NaN
2024-06-27,57.16,59.51,56.77,59.20,33846,73.33430,57.9254,26429.2,10.81,28.062900,1,True,True,False,True,True,5,NaN
2024-07-26,58.28,59.28,57.42,59.28,21820,72.27505,58.8060,28224.2,-22.13,-22.690457,-3,True,True,False,True,True,5,NaN
2024-07-31,57.30,59.11,56.94,58.70,20492,72.08275,58.5948,25091.4,-13.28,-18.330583,-6,True,True,False,True,True,5,NaN
2024-08-15,59.45,61.00,55.88,59.30,31276,71.69800,58.0256,31683.0,15.23,-1.284601,1,True,True,False,True,True,5,NaN


# 9. Final Watch List